### 피봇 테이블과 그룹 분석

피봇 테이블 (pivot table)
- 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용해서
- 데이터를 조회하여 펼쳐놓은 것
- 행과 열 인덱스만 보아도 어느 열의 어느 인덱스인지 쉽게 알 수 있음

Pandas는 피봇 테이블을 만들기 위한 pivot() 메소드 제공
- 첫 번째 인수는 행 인덱스로 사용할 열 이름
- 두 번째 인수는 열 인덱스로 사용할 열 이름
- 마지막 데이터로 사용할 열 이름
- pivot(행 인덱스 값, 열 인덱스 값, 데이터 열)

피봇 처리 과정
- Pandas는 지정된 두 열을 각각 행 인덱스와 열 인덱스로 바꾼 후
- 행 인덱스의 라벨 값이 첫 번째 키의 값과 같고 열 인덱스의 라벨 값이 두 번재 키의 값과 같은 데이터를 찾아서 해당 칸에 넣는다
- 만일 주어진 데이터가 존재하지 않으면 해당 칸에 NaN을 넣는다

In [2]:
import pandas as pd
import numpy as np

In [7]:
data = {
    '도시' : ['서울','서울','서울','부산','부산','부산','인천','인천'],
    '연도' : ['2015','2010','2005','2015','2010','2005','2015','2010'],
    '인구' : [9878748, 9756424, 9345214, 5421213, 5321451, 5120211, 4212121, 4111212],
    '지역' : ['수도권','수도권','수도권','경상권','경상권','경상권','수도권','수도권']
}

columns = ['도시','연도','인구','지역']

df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9878748,수도권
1,서울,2010,9756424,수도권
2,서울,2005,9345214,수도권
3,부산,2015,5421213,경상권
4,부산,2010,5321451,경상권
5,부산,2005,5120211,경상권
6,인천,2015,4212121,수도권
7,인천,2010,4111212,수도권


행과 열 인덱스만 보면 어떤 도시의 어떤 시점의 인구인지 쉽게 알 수 있도록 피봇 테이블로 작성 : pivot() 메소드 사용
    - 행 인덱스 : 도시
    - 열 인덱스 : 연도
    - 데이터 : 인구 (인구 수)

In [8]:
# 피봇 테이블 작성
# pivot(행 인덱스, 열 인덱스, 데이터 열)
df1.pivot('도시','연도','인구')
# 없는 값은 NaN으로 채움

연도,2005,2010,2015
도시,,,
부산,5120211.0,5321451.0,5421213.0
서울,9345214.0,9756424.0,9878748.0
인천,NaN,4111212.0,4212121.0


피봇 테이블은 set_index() 메소드와 unstack() 메소드를 사용해서 만들 수도 있음

In [10]:
# set_index() 사용
df1.set_index(['도시','연도'])[['인구']].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  5120211.0  5321451.0  5421213.0
서울  9345214.0  9756424.0  9878748.0
인천        NaN  4111212.0  4212121.0

행 인덱스와 열 인덱스는 데이터를 찾는 키(key) 역할 수행
따라서 키 값으로 데이터가 단 하나만 찾아져야 하는데
행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러 발생
- 키가 잘못 설정된 것임

In [11]:
df1
# '지역','연도'를 키로 한다면 데이터가 2개 이상 존재 --- 에러
# 0행 : 2015 수도권
# 6행 : 2015 수도권

,도시,연도,인구,지역
0,서울,2015,9878748,수도권
1,서울,2010,9756424,수도권
2,서울,2005,9345214,수도권
3,부산,2015,5421213,경상권
4,부산,2010,5321451,경상권
5,부산,2005,5120211,경상권
6,인천,2015,4212121,수도권
7,인천,2010,4111212,수도권


In [13]:
# 지역과 연도를 키로 설정할 경우
# df1.pivot('지역','연도','인구') --- 에러(ValueError) 발생 !

In [14]:
# 예외 처리하고 에러 내용 확인
try :
    df1.pivot('지역','연도','인구')
except ValueError as e :
    print('Error : ',e)

Error :  Index contains duplicate entries, cannot reshape


### 그룹 분석

만일 키가 지정하는 조건에 맞는 데이터가 2개 이상이라서
데이터 그룹을 이루는 경우에는 그룹의 특서을 보여주는 그룹 분석을 수행

그룹 분석은 피봇 테이블과 달리 키에 의해서 결정되는 것이 아니라,
데이터가 여러 개 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산함

Pandas 에서는 groupby() 메소드 사용해서 그룹 분석 수행

그룹 분석 순서
(1) 분석하고자 하는 시리즈나 데이터프레임에 groupby() 메소드 호출해서 그룹화 하고,
(2) 그룹 객체에 대해 그룹 연산 수행

groupby() 메소드
- 데이터를 그룹별로 분류하는 역할
- groupby() 메소드 인수
    - 열 또는 열 리스트
    - 행 인덱스
- 연산 결과로 그룹 데이터를 나타내는 GroupBy 클래스 객체 반환
- 객체에는 그룹별로 연산할 수 있는 그룹 연산 메소드가 포함(sum()/mean())

그룹 연산 메소드
GroupBy 클래스 객체 뒤에 붙일 수 있는 그룹 연산 메소드
- size(), count() : 그룹 데이터의 개수
    - count() : Null 값이 아닌 행만 센다
    - size() : Null 값인 행도 모두 포함해서 센다
- mean(), median(), min(), max() : 그룹 데이터의 평균, 중앙값, 최소값, 최대값
- sum(), prod(), std(), var(), quantile() : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
- first(), last() : 그룹 데이터 중 가장 첫 번째, 마지막 데이터

In [15]:
# 그룹 연산 수행할 데이터프레임 생성
np.random.seed(0)

df2 = pd.DataFrame({
    'key1' : ['A','A','B','B','A'],
    'key2' : ['one','two','one','two','one'],
    'data1' : [1,2,3,4,5],
    'data2' : [10,20,30,40,50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


예) df2에서 key1의 값(A 또는 비)에 다른 data1의 평균, 합계는?
groupby() 메소드를 사용해서 그룹 A와 그룹 B로 구분해서 데이터를 만든다

In [17]:
groups = df2.groupby(df2.key1)
groups # groupby 객체 반환

In [18]:
groups = df2.groupby(df2['key1'])
groups

In [21]:
# groupby 클래스 객체의 groups 속성 확인
groups.groups

{'A': Int64Index([0, 1, 4], dtype='int64'),
 'B': Int64Index([2, 3], dtype='int64')}

In [22]:
# groupby 객체에는 그룹별로 연산할 수 있는 그룹 연산 메소드가 포함되어 있음
# sum() 메소드를 사용해서 그룹별 합계를 구하기
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [24]:
# 중간에서 groupby 객체를 명시적으로 사용할 필요가 없다면
# groupby() 메소드와 그룹 연산 메소드를 연속으로 호출해서 사용 가능
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [25]:
# 데이터 그룹으로 나눈 groupby 객체 또는 그룹 분석한 결과에서 data1만 추출 가능
df2.groupby(df2.key1)['data1'].sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [26]:
# 전체 데이터를 분석한 후 data1만 추출 가능
df2.groupby(df2.key1).sum()['data1']

key1
A    8
B    7
Name: data1, dtype: int64

In [29]:
# groupby 객체에서 data2만 선택해서 분석
df2.groupby(df2.key1)['data2'].sum()

key1
A    80
B    70
Name: data2, dtype: int64

In [28]:
# 전체 데이터를 분석한 후 data2만 선택
df2.groupby(df2.key1).sum()['data2']

key1
A    80
B    70
Name: data2, dtype: int64

In [32]:
# key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를
# 시리즈가 아닌 데이터프레임으로 구하기
df2.groupby(df2.key1)[['data1']].sum()

,data1
key1,
A,8
B,7


In [33]:
# 인덱스를 설정하지 않고 모두 열로 만들 경우
# as_index=False 로 설정
df2.groupby(df2.key1, as_index=False)['data1'].sum()

,key1,data1
0,A,8
1,B,7


복합키 (key1, key2) 사용하는 경우
- 사용하고자 하는 키가 여러 개면 리스트 이용

In [38]:
df2.data1.groupby([df2.key1, df2.key2]).sum()
# == df2.data1.groupby([df2['key1'], df2['key2']]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

pivot_table()
- pivot() 메소드와 groupby() 메소드의 중간 성격을 가지고 있음
- groupby() 메소드처럼 그룹 분석을 수행하지만 최종적으로는 pivot() 메소드처럼 피봇 테이블을 생성
- 즉, groupby() 명령의 결과에 unstack() 기능을 자동 적용하여 2차원 형태로 변형

pivot_table(data, values, index, columns, aggfunc, fill_value, margins, margins_name)
- data : 분석할 데이터
- values : 분석할 데이터프레임에서 분석할 열
- index : 행 인덱스로 들어갈 키 열 또는 열 리스트
- columns : 열 인덱스로 들어갈 키 열 또는 리스트
- aggfunc : 분석 메소드
- fill_value : NaN 대체값
- magins : 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
- margins_name : 마진 열(행)의 이름

주로 사용하는 형태
- pivot_tagle(데이터, 행 인덱스, 열 인덱스)

In [39]:
df1

,도시,연도,인구,지역
0,서울,2015,9878748,수도권
1,서울,2010,9756424,수도권
2,서울,2005,9345214,수도권
3,부산,2015,5421213,경상권
4,부산,2010,5321451,경상권
5,부산,2005,5120211,경상권
6,인천,2015,4212121,수도권
7,인천,2010,4111212,수도권


In [40]:
# pivot_table(데이터, 행 인덱스, 열 인덱스)
df1.pivot_table('인구','도시','연도')

연도,2005,2010,2015
도시,,,
부산,5120211.0,5321451.0,5421213.0
서울,9345214.0,9756424.0,9878748.0
인천,NaN,4111212.0,4212121.0


margins=True 인수를 설정하면 aggfunc으로 주어진 분석 방법을 다음과 같이 계산한다.
- 해당 열의 모든 데이터 (예 : 열 합계, 열 평균)
- 해당 행의 모든 데이터 (예 : 행 합계, 행 평균)
- 전체 데이터에 대해 적용한 결과를 같이 보여줌 (전체 합계, 전체 평균)
- aggfunc 인수가 주어지지 않으면 디폴트로 평균 계산

In [45]:
df1.pivot_table('인구','도시','연도', margins=True, margins_name='평균')
# appfunc-mean 생략 (디폴트)

연도,2005,2010,2015,평균
도시,,,,
부산,5120211.0,5.321451e+06,5.421213e+06,5.287625e+06
서울,9345214.0,9.756424e+06,9.878748e+06,9.660129e+06
인천,NaN,4.111212e+06,4.212121e+06,4.161666e+06
평균,7232712.5,6.396362e+06,6.504027e+06,6.645824e+06


In [44]:
df1['인구'].mean() # 인구수 전체 평균 : '인구' 열 평균

6645824.25

In [50]:
# 각 행의 합계, 각 열의 합계, 전체 합계 구하기
# 열 이름 : 합계
df1.pivot_table('인구','도시','연도', aggfunc='sum', margins=True, margins_name = '합계')

연도,2005,2010,2015,합계
도시,,,,
부산,5120211.0,5321451.0,5421213.0,15862875
서울,9345214.0,9756424.0,9878748.0,28980386
인천,NaN,4111212.0,4212121.0,8323333
합계,14465425.0,19189087.0,19512082.0,53166594


In [52]:
# pivot_table()에서 인덱스 값으로 리스트를 넣으면 다중 인덱스 테이블 생성
df1.pivot_table('인구',index=['연도','도시'])

인구
연도   도시         
2005 부산  5120211
     서울  9345214
2010 부산  5321451
     서울  9756424
     인천  4111212
2015 부산  5421213
     서울  9878748
     인천  4212121

### Tip 데이터 예제

식당에서 식사 후에 내는 팁(tip)과 관련된 데이터를 이용하여
좀 더 구체적으로 그룹 분석 방법 수행
Seaborn 패키지에 포함되어 있는 tips 데이터 세트 사용
tips 데이터 세트의 열 
- total_bill : 식사 대금
- tip : 팁
- sex : 성별
- smoker : 흡연/금연 여부
- day : 요일
- time : 시간
- size : 인원

In [53]:
import seaborn as sns

In [58]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [55]:
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


분석 목표
- 식사 대금 대비 팁의 비율이 어떤 경우에 가장 높아지는지 찾는 것

In [101]:
#(1) 식사 대금과 팁의 비율을 나타내는 tip_pct 열 추가
tips['tip_pct'] = tips.tip / tips.total_bill
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [83]:
#(2) 성별로 나누어 개수 확인
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [102]:
tips.groupby('sex').size() # 위와 동일

sex
Male      157
Female     87
dtype: int64

In [103]:
#(3) 성별, 흡연유무로 나누어 데이터 개수 확인
tips.groupby(['sex','smoker']).size() # count()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [88]:
#(4) 피봇 테이블 형태로 변환
# 흡연 비율 데이터를 대상으로 성별, 흡연유무로 나누어서 개수를 세어서
# 행, 열, 전체 수로 표시
tips.pivot_table('tip_pct','sex','smoker', aggfunc='count', margins=True, margins_name = 'total')

smoker,Yes,No,total
sex,,,
Male,60,97,157
Female,33,54,87
total,93,151,244


In [92]:
#(5) (4)번에 평균으로 출력
tips.pivot_table('tip_pct','sex','smoker', aggfunc='mean', margins=True, margins_name = 'avg') # aggfunc = 'mean' 생략 가능

smoker,Yes,No,avg
sex,,,
Male,0.152771,0.160669,0.157651
Female,0.182150,0.156921,0.166491
avg,0.163196,0.159328,0.160803


In [98]:
#(6) 성별과 흡연유무에 따른 평균 팁 비율 확인(출력)
tips.pivot_table('tip_pct','sex','smoker')

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [104]:
# 다른 통계값 확인
tips.groupby(['sex','smoker'])[['tip_pct']].describe()

tip_pct                                                    \
                count      mean       std       min       25%       50%   
sex    smoker                                                             
Male   Yes       60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       No        97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
Female Yes       33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
       No        54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
sex    smoker                      
Male   Yes     0.191697  0.710345  
       No      0.186220  0.291990  
Female Yes     0.198216  0.416667  
       No      0.181630  0.252672

In [189]:
# 연습 문제 1
# 타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.
titanic = sns.load_dataset("titanic")

# (1) qcut 명령으로 세 개의 나이 그룹을 만든다.
# (2) 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
#     행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.
# (3) 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.

In [109]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [126]:
# titanic.age.max() # 80

In [190]:
# (1) qcut 명령으로 세 개의 나이 그룹을 만든다.
bins = [0,20,65,100]
titanic['age_ctg']=pd.cut(titanic.age, bins, labels = ['minor','adult','senior'])
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_ctg
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,minor
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult


In [193]:
# (2) 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
#     행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.

In [191]:
# 생존자를 성별, 선실, 나이로 그룹화
titanic.groupby(['sex','age_ctg','class'])[['survived']]

In [192]:
titanic.groupby(['sex','age_ctg','class'])[['survived']].sum()

survived
sex    age_ctg class           
female minor   First       13.0
               Second      16.0
               Third       24.0
       adult   First       69.0
               Second      52.0
               Third       23.0
       senior  First        NaN
               Second       NaN
               Third        NaN
male   minor   First        4.0
               Second      10.0
               Third       15.0
       adult   First       35.0
               Second       5.0
               Third       23.0
       senior  First        1.0
               Second       0.0
               Third        0.0

In [194]:
titanic.groupby(['sex','age_ctg','class'])[['survived']].count() # 총합계

survived
sex    age_ctg class           
female minor   First       14.0
               Second      16.0
               Third       47.0
       adult   First       71.0
               Second      58.0
               Third       55.0
       senior  First        NaN
               Second       NaN
               Third        NaN
male   minor   First        7.0
               Second      19.0
               Third       76.0
       adult   First       90.0
               Second      78.0
               Third      175.0
       senior  First        4.0
               Second       2.0
               Third        2.0

In [195]:
# 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
titanic.groupby(['sex','age_ctg','class'])[['survived']].sum()/titanic.groupby(['sex','age_ctg','class'])[['survived']].count()

survived
sex    age_ctg class           
female minor   First   0.928571
               Second  1.000000
               Third   0.510638
       adult   First   0.971831
               Second  0.896552
               Third   0.418182
       senior  First        NaN
               Second       NaN
               Third        NaN
male   minor   First   0.571429
               Second  0.526316
               Third   0.197368
       adult   First   0.388889
               Second  0.064103
               Third   0.131429
       senior  First   0.250000
               Second  0.000000
               Third   0.000000

In [197]:
# (3) 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.
# 인수 값을 순서대로 입력하면 인수 지정하지 않아도 됨
titanic.pivot_table('survived','sex','class', aggfunc='mean')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [198]:
titanic.pivot_table('survived',['sex','class'],'age_ctg', aggfunc='mean')

age_ctg           minor     adult  senior
sex    class                             
female First   0.928571  0.971831     NaN
       Second  1.000000  0.896552     NaN
       Third   0.510638  0.418182     NaN
male   First   0.571429  0.388889    0.25
       Second  0.526316  0.064103    0.00
       Third   0.197368  0.131429    0.00

In [199]:
titanic.pivot_table('survived',['sex','age_ctg'],'class', aggfunc='mean')

class              First    Second     Third
sex    age_ctg                              
female minor    0.928571  1.000000  0.510638
       adult    0.971831  0.896552  0.418182
male   minor    0.571429  0.526316  0.197368
       adult    0.388889  0.064103  0.131429
       senior   0.250000  0.000000  0.000000